In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score

import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification



data_train = pd.read_csv("training.csv")

columns_to_filter = data_train.columns[13:]

data_train

count_na = pd.DataFrame(data_train.isna().sum())

with pd.option_context('display.max_rows', None,):
    print(count_na)

data_train_dropna = data_train.dropna(subset = columns_to_filter)

columns = ['patient_race','bmi','metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type' ]
data_train_dropna_part2 = data_train_dropna.drop(columns, axis=1)

data_train_dropna_part2 = data_train_dropna_part2.fillna(value="UNKNOWN")

data_train_dropna_part2

count_na_2 = pd.DataFrame(data_train_dropna_part2.isna().sum())

with pd.option_context('display.max_rows', None,):
    print(count_na_2)

columns_object = data_train_dropna_part2.dtypes[data_train_dropna_part2.dtypes == "object"]

columns_object.index

# Create dummy variables for categorical columns
data_train_dropna_part3  = pd.get_dummies(data_train_dropna_part2, columns=['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],  prefix = ['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'], prefix_sep='_')

from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
data_train_dropna_part4 =  pd.DataFrame(scaler.fit_transform(data_train_dropna_part3), columns=data_train_dropna_part3.columns)

data_train_dropna_part4

data_train_dropna_part4_x = data_train_dropna_part4.drop("DiagPeriodL90D", axis = 1).drop("patient_id", axis = 1).drop("patient_zip3", axis = 1).reset_index(drop = True)

data_train_dropna_part4_y = data_train_dropna_part4[["DiagPeriodL90D"]]

X_train, X_test, y_train, y_test = train_test_split(data_train_dropna_part4_x  ,data_train_dropna_part4_y,  stratify = data_train_dropna_part4_y,
                                                    train_size = 0.8, random_state = 20240114)

                                           0
patient_id                                 0
patient_race                            6385
payer_type                              1803
patient_state                             51
patient_zip3                               0
patient_age                                0
patient_gender                             0
bmi                                     8965
breast_cancer_diagnosis_code               0
breast_cancer_diagnosis_desc               0
metastatic_cancer_diagnosis_code           0
metastatic_first_novel_treatment       12882
metastatic_first_novel_treatment_type  12882
Region                                    52
Division                                  52
population                                 1
density                                    1
age_median                                 1
age_under_10                               1
age_10_to_19                               1
age_20s                                    1
age_30s   

In [ ]:
#coba gridsearch tanpa PCA #current best
xgb_tuned = xgb.XGBClassifier(random_state=1)

# Grid of parameters to choose from

parameters = {
    "n_estimators": [10, 50, 100],
    "subsample":[0.6, 0.8, 1],
    "learning_rate":[0.01, 0.1, 0.5, 1],
    "gamma":[0.01, 0.1, 1, 5],
    "colsample_bytree":[0.5, 0.7, 0.9, 1],
    "alpha":[0, 0.1, 0.5]
}



# Run the grid search
grid_obj = GridSearchCV(xgb_tuned, parameters, scoring="roc_auc",
                        cv=5, n_jobs=-1, verbose=2)

grid_obj = grid_obj.fit(X_train, y_train)

# Predict probabilities on the test set
y_pred_proba = grid_obj.predict_proba(X_test)

# Compute ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("ROC AUC Score:", roc_auc)


data_test = pd.read_csv("test.csv")

columns = ['patient_race','bmi','metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type' ]
data_test_dropna_part2 = data_test.drop(columns, axis=1)

object_columns = data_test_dropna_part2 .select_dtypes(include=['object']).columns

# Fill NaN values only in object columns
data_test_dropna_part2 [object_columns] = data_test_dropna_part2 [object_columns].fillna('UNKNOWN')

object_columns = data_test_dropna_part2 .select_dtypes(include=['object']).columns

# Fill NaN values only in object columns
data_test_dropna_part2 [object_columns] = data_test_dropna_part2 [object_columns].fillna('UNKNOWN')

# Identify numeric columns
numeric_columns = data_test_dropna_part2.select_dtypes(include=['float', 'int']).columns

# Fill NaN values only in numeric columns
data_test_dropna_part2[numeric_columns] = data_test_dropna_part2[numeric_columns].fillna(0)  # You can replace 0 with any value you want


# Create dummy variables for categorical columns
data_test_dropna_part3  = pd.get_dummies(data_test_dropna_part2, columns=['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],  prefix = ['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'], prefix_sep='_')


# fit and transform the data
data_test_dropna_part4 =  pd.DataFrame(scaler.fit_transform(data_test_dropna_part3), columns=data_test_dropna_part3.columns)

data_test_dropna_part4_x = data_test_dropna_part4.drop("patient_id", axis = 1).drop("patient_zip3", axis = 1)

data_test_dropna_part4_x  = data_test_dropna_part4_x.reindex(columns=data_train_dropna_part4_x.columns, fill_value=0) 
result = pd.concat([pd.DataFrame(data_test['patient_id']),pd.DataFrame(grid_obj.predict_proba(data_test_dropna_part4_x))[1]], axis = 1).rename(columns = { 1:"DiagPeriodL90D"})

result['patient_id'] = result['patient_id'].astype(int)

#result.to_csv("output.csv", index = False)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits
